In [1]:
import sqlite3
import pandas as pd
import numpy as np

In [2]:
conn = sqlite3.connect('finra.db')
cursor = conn.cursor()

In [56]:
frame = cursor.execute("SELECT disclosures.Allegations FROM disclosures WHERE disclosures.'Broker Comment' LIKE '%INFORMAL%'").fetchall()

In [43]:
import io
import requests
from PyPDF2 import PdfFileReader

# filename = Path('metadata.pdf')
response = requests.get('https://files.brokercheck.finra.org/firm/firm_361.pdf')
f = io.BytesIO(response.content)
reader = PdfFileReader(f)
reader.decrypt()
reader.getNumPages()

contents = reader.getPage(0).extractText().split('\n')
# with open('metadata.pdf', 'wb') as f:
#     f.write(response.content)

# filename.write_bytes(response.content)

TypeError: decrypt() missing 1 required positional argument: 'password'

In [30]:
filename

WindowsPath('metadata.pdf')

In [25]:
from pdfminer.layout import LAParams
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.converter import PDFPageAggregator
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.layout import LTTextBoxHorizontal
from pdfminer.layout import LTPage

def doc_read(path):
    text_list = []

    document = open(path, 'rb')
#     try:    #Create resource manager
    rsrcmgr = PDFResourceManager()
    # Set parameters for analysis.
    laparams = LAParams()
    # Create a PDF page aggregator object.
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
#     print(PDFPage.get_pages(document)[0])
    for page in PDFPage.get_pages(document):
        interpreter.process_page(page)
        # receive the LTPage object for the page.
        layout = device.get_result()
        for element in layout:
            try:
                text_list.append(element.get_text().replace('\n',''))
            except:
                pass

    document.close()
#     except Exception as e:
#         document.close()
#         print(e)
    return text_list

In [26]:
from pdfminer.converter import TextConverter
from io import StringIO
path = 'metadata.pdf'
rsrcmgr = PDFResourceManager()
retstr = StringIO()
codec = 'utf-8'
laparams = LAParams()
device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
fp = open(path, 'rb')
interpreter = PDFPageInterpreter(rsrcmgr, device)
password = ""
maxpages = 0
caching = True
pagenos=set()
for page in PDFPage.get_pages(fp, pagenos, password=password,maxpages=maxpages,caching=caching, check_extractable=True):
    interpreter.process_page(page)

error: argument out of range